<a href="https://colab.research.google.com/github/youssefbelkhiri/machine_learning_models/blob/main/tempurature_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest Regression

## Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [ ]:
dataset = pd.read_csv('GlobalLandTemperaturesByMajorCity.csv')
dataset.shape

(239177, 7)

In [ ]:
dataset.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

# Correlation Visualization befor data preprocessing

# **Data Pre-processing**

In [ ]:
dataset.isnull().sum()

dt                                   0
AverageTemperature               11002
AverageTemperatureUncertainty    11002
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [ ]:
def fill_missing_temp(group):
    group['AverageTemperature'] = group['AverageTemperature'].fillna(group['AverageTemperature'].mean())
    return group
def wrangle(data):
  data = data.copy()
  data = data.drop(columns = ['AverageTemperatureUncertainty','Country','Latitude' , 'Longitude'],axis=1)
  data["dt"] = pd.to_datetime(data["dt"])
  data["month"] = data["dt"].dt.month
  data["year"] = data["dt"].dt.year
  data = data.drop("dt",axis=1)
  data = data[data["year"]>=1950]
  data = data.groupby(['City', data["month"]]).apply(fill_missing_temp)
  return data


In [ ]:
dataset = wrangle(dataset)
dataset.head()

<ipython-input-34-94da3dd67f33>:12: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.groupby(['City', data["month"]]).apply(fill_missing_temp)


,AverageTemperature,City,month,year
1212,26.773,Abidjan,1,1950
1213,27.527,Abidjan,2,1950
1214,28.344,Abidjan,3,1950
1215,27.830,Abidjan,4,1950
1216,26.896,Abidjan,5,1950


In [ ]:
dataset.shape

(76500, 4)

In [ ]:
dataset.isnull().sum()

AverageTemperature    0
City                  0
month                 0
year                  0
dtype: int64

# Correlation Visualization after preprocessing

# Train-Test Split

In [ ]:
y = dataset["AverageTemperature"]
x = dataset[["City" , "month" , "year"]]

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_x= LabelEncoder()
x["City"]= labelencoder_x.fit_transform(x["City"])
t = len(x["City"].unique())
onehotencoder= OneHotEncoder()
x= onehotencoder.fit_transform(x).toarray()

<ipython-input-39-80c10dbb5526>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["City"]= labelencoder_x.fit_transform(x["City"])


In [ ]:
print(x)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
from sklearn.model_selection import train_test_split
x_t , x_te , y_t , y_te = train_test_split(x,y,test_size=0.2,random_state=42)

# Establish Baseline Mean Absolute Error

In [ ]:
from sklearn.metrics import mean_absolute_error
y_p = [y_t.mean()] * len(y_t)
print('MAE:' , round(mean_absolute_error(y_t,y_p),5))

MAE: 7.62358


## Linear Regression Model
> Bloc en retrait



In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
forest = make_pipeline (
    SelectKBest(k='all'),
    RandomForestRegressor(n_estimators=100,max_depth=50,random_state=77,n_jobs=-1)
)
forest.fit(x_t,y_t)

Pipeline(steps=[('selectkbest', SelectKBest(k='all')),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=50, n_jobs=-1,
                                       random_state=77))])

## Predicting a new result

In [ ]:
from sklearn.metrics import mean_absolute_error
print('linear regression training MAE:',round(mean_absolute_error(y_t , forest.predict(x_t)), 5))
print('linear regression test MAE:',round(mean_absolute_error(y_te , forest.predict(x_te)), 5))

linear regression training MAE: 1.75278
linear regression test MAE: 2.0376


In [ ]:
y_p = forest.predict(x_te)

In [ ]:
errors = abs(y_p - y_te)
mape = 100 * (errors/y_t)
accuracy = 100 - np.mean(mape)
print('linear regression model prediction accuracy:' , round(mape , 2),'%')

linear regression model prediction accuracy: 1212     NaN
1213     NaN
1214     NaN
1215     NaN
1216     NaN
          ..
239172   NaN
239173   NaN
239174   NaN
239175   NaN
239176   NaN
Name: AverageTemperature, Length: 76500, dtype: float64 %


## Visualising the Random Forest Regression results (higher resolution)

In [ ]:
y_p = forest.predict(x_te)
print("train Score:",forest.score(x_t,y_t))
print("test Score:",forest.score(x_te,y_te))

train Score: 0.9346610959982136
test Score: 0.9221451343393431
